In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score

In [ ]:
df_test = pd.read_csv('../input/evaluation/test_data.csv')
df_eval = pd.read_csv('../input/evaluation/final_submission.csv')

In [ ]:
df_test.drop(columns=['Unnamed: 0','index'], inplace=True)
df_eval.drop(columns=['Unnamed: 0'], inplace=True)
df_eval.columns = ['ImageId', 'EncodedPixels', 'Category']

df_eval_unique = df_eval.drop_duplicates()

In [ ]:
df = df_test.merge(df_eval_unique, on=['ImageId', 'Category'], how='left', suffixes=('_test', '_eval'))
df.rename(columns={"Category" : "Category_test"}, inplace=True)

In [ ]:
df['Category_eval']=df['Category_test']

In [ ]:
df.loc[pd.isna(df['EncodedPixels_eval']), 'Category_eval'] = 500 # dummy class for objects where class were not predicted

In [ ]:
df.fillna('1 1', inplace=True)

In [ ]:
def rle_decode(mask_rle, shape):
    shape = (shape[1], shape[0])
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape[1]*shape[0]) 

In [ ]:
df['EncodedPixels_eval'] = df['EncodedPixels_eval'].apply(lambda x: " ".join([str(int(float(i))) for i in x.split()]))

In [ ]:
df['IoU'] = 0

Jaccard similarity coefficient, defined as the size of the intersection divided by the size of the union of two label sets, is used to compare set of predicted labels for a sample to the corresponding set of labels in y_true

In [ ]:
df.loc[df['Category_eval']!=500, 'IoU'] = df.apply(lambda x: jaccard_score(rle_decode(x['EncodedPixels_test'], (x['Height'], x['Width'])), 
                                                        rle_decode(x['EncodedPixels_eval'], (x['Height'], x['Width']))), axis=1)

In [ ]:
df.to_csv('results.csv', index=False)

In [ ]:
print('Overall accuracy: ', accuracy_score(df['Category_test'], df['Category_eval']))

In [ ]:
print('Average IoU: ', df['IoU'].mean())